### ===My Notes=== ###

This tutorial is [from this page](https://huggingface.co/docs/transformers/en/training)

### ===End My Notes=== ###

In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Fine-tune a pretrained model

There are significant benefits to using a pretrained model. It reduces computation costs, your carbon footprint, and allows you to use state-of-the-art models without having to train one from scratch. 🤗 Transformers provides access to thousands of pretrained models for a wide range of tasks. When you use a pretrained model, you train it on a dataset specific to your task. This is known as fine-tuning, an incredibly powerful training technique. In this tutorial, you will fine-tune a pretrained model with a deep learning framework of your choice:

* Fine-tune a pretrained model with 🤗 Transformers [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer).
* Fine-tune a pretrained model in TensorFlow with Keras.
* Fine-tune a pretrained model in native PyTorch.

<a id='data-processing'></a>

## Prepare a dataset

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/_BZearw7f0w?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

Before you can fine-tune a pretrained model, download a dataset and prepare it for training. The previous tutorial showed you how to process data for training, and now you get an opportunity to put those skills to the test!

Begin by loading the [Yelp Reviews](https://huggingface.co/datasets/yelp_review_full) dataset:

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

### ===My Notes=== ##

**Questions:** How can we tell what splits are available by looking at the dataset object?  You could look at the data card, but in the Yelp case this doesn't give a formal breakdown.

The [.load_dataset()](https://huggingface.co/docs/datasets/v2.18.0/en/package_reference/loading_methods#datasets.load_dataset) function is described at the link.  Note that it returns a [Dataset](https://huggingface.co/docs/datasets/v2.18.0/en/package_reference/main_classes#datasets.Dataset) or a [DataSetDict](https://huggingface.co/docs/datasets/v2.18.0/en/package_reference/main_classes#datasets.DatasetDict). This has TONS of methods--dealing with lots of data formats. The one I am interested in is a DatasetDict, which is a dictionary so you can use the ".keys()" method to find out what it contains.  (See the top of the DatasetDict page.)


In [5]:
dataset.dtype  # Learning the hard way this is a DatasetDict

AttributeError: 'DatasetDict' object has no attribute 'dtype'

In [6]:
dataset.keys()

dict_keys(['train', 'test'])

In [7]:
dataset["train"][:5]

{'label': [4, 1, 3, 3, 0],
 'text': ["dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
  "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You 

### ===End My Notes=== ###

As you now know, you need a tokenizer to process the text and include a padding and truncation strategy to handle any variable sequence lengths. To process your dataset in one step, use 🤗 Datasets [`map`](https://huggingface.co/docs/datasets/process.html#map) method to apply a preprocessing function over the entire dataset:

### ===My Notes=== ###

**Note:**  A Tokenizer is really about general preprocessing--not just creating tokens.  "Tokenizers are one of the core components of the NLP pipeline. They serve one purpose: to translate text into data that can be processed by the model."  [Here](https://medium.com/@awaldeep/hugging-face-understanding-tokenizers-1b7e4afdb154) is a NICE reference. Note in particular this comment:

"After normalization and pre-processing steps, we apply a training algorithm to the text data. This output of this step is dependent on the type of training strategy we are going to use. The state-of-the-art models use subword tokenization algorithms, for example BERT uses WordPiece tokenization, GPT, GPT-2 use BPE, AIBERT uses unigram etc."

What is 'AutoTokenizer'?  Here is the page on [Auto Classes]( https://huggingface.co/docs/transformers/v4.38.2/en/model_doc/auto#auto-classes). See the bar on the RIGHT side and note there are plenty of "Auto" classes.  In general, from the documentation, "In many cases, the architecture you want to use can be guessed from the name or the path of the pretrained model you are supplying to the from_pretrained() method. AutoClasses are here to do this job for you so that you automatically retrieve the relevant model given the name/path to the pretrained weights/config/vocabulary."

Here is the discussion of [AutoTokenizer](https://huggingface.co/docs/transformers/v4.38.2/en/model_doc/auto#transformers.AutoTokenizer), which explains how it is constructed using ".from_pretrained()".  Note it instantiates based on "the model_type property of the config object" or something similar. 

Not that 'gpt2' (etc) is one of the classes it can figure out, based on the list above.  **BUT** the example below uses ".from_pretrained(bert-base_cased)", which is a HuggingFace model.

The DatasetDict.map() function is described [here](https://huggingface.co/docs/datasets/v2.18.0/en/package_reference/main_classes#datasets.DatasetDict.map).

**Question:** What sort of Tokenizer does this return, and what do the methods do? More specifically, 

Recall that Tokenizers are [discussed here](https://huggingface.co/docs/transformers/main_classes/tokenizer).

### ===End My Notes=== ###

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

If you like, you can create a smaller subset of the full dataset to fine-tune on to reduce the time it takes:

In [3]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

<a id='trainer'></a>

## Train

At this point, you should follow the section corresponding to the framework you want to use. You can use the links
in the right sidebar to jump to the one you want - and if you want to hide all of the content for a given framework,
just use the button at the top-right of that framework's block!

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/nvBXf7s7vTI?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

## Train with PyTorch Trainer

🤗 Transformers provides a [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) class optimized for training 🤗 Transformers models, making it easier to start training without manually writing your own training loop. The [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) API supports a wide range of training options and features such as logging, gradient accumulation, and mixed precision.

Start by loading your model and specify the number of expected labels. From the Yelp Review [dataset card](https://huggingface.co/datasets/yelp_review_full#data-fields), you know there are five labels:

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<Tip>

You will see a warning about some of the pretrained weights not being used and some weights being randomly
initialized. Don't worry, this is completely normal! The pretrained head of the BERT model is discarded, and replaced with a randomly initialized classification head. You will fine-tune this new model head on your sequence classification task, transferring the knowledge of the pretrained model to it.

</Tip>

### ===My Notes=== ###

What can the Huggingface Trainer class do?

Here is the [main documentation page](https://huggingface.co/docs/transformers/main_classes/trainer)


AutoModelForSequenceClassification is found under the "Auto Classes" that we referenced above.  See [this page](https://huggingface.co/docs/transformers/model_doc/auto).  The idea is the same:  It figures out what model you have and it does this (e.g., builds the head) so that you can fine-tune the model.  See [this page](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForSequenceClassification) for this particular AutoModel.  The key idea is that it will "Instantiate one of the model classes of the library (with a sequence classification head) from a pretrained model."

Note there is a special [GPT2ForSequenceClassification](https://huggingface.co/docs/transformers/v4.38.2/en/model_doc/gpt2#transformers.GPT2ForSequenceClassification), which is described as "The GPT2 Model transformer with a sequence classification head on top (linear layer)." Also interesting: "This model is also a PyTorch torch.nn.Module subclass. Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general usage and behavior."  In particular, it has a '.forward()' method.  **Question:** How is this different from the more general AutoModelFor... classs?  Will the more general model just pass it on to this on the basis of what model type you feed it?

The ".from_pretrained()" method actually loads the weights from the model--instead of just instantiating it. 

### ===End My Notes=== ###

### Training hyperparameters

Next, create a [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) class which contains all the hyperparameters you can tune as well as flags for activating different training options. For this tutorial you can start with the default training [hyperparameters](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments), but feel free to experiment with these to find your optimal settings.

Specify where to save the checkpoints from your training:

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

### ===My Notes=== ###

The documentation page is [here](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments).  There are a HUGE number of parameters you could feed to it--as you might expect.
### ===End My Notes=== ###

### Evaluate

[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) does not automatically evaluate model performance during training. You'll need to pass [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) a function to compute and report metrics. The [🤗 Evaluate](https://huggingface.co/docs/evaluate/index) library provides a simple [`accuracy`](https://huggingface.co/spaces/evaluate-metric/accuracy) function you can load with the [evaluate.load](https://huggingface.co/docs/evaluate/main/en/package_reference/loading_methods#evaluate.load) (see this [quicktour](https://huggingface.co/docs/evaluate/a_quick_tour) for more information) function:

In [6]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Call `compute` on `metric` to calculate the accuracy of your predictions. Before passing your predictions to `compute`, you need to convert the predictions to logits (remember all 🤗 Transformers models return logits):

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

If you'd like to monitor your evaluation metrics during fine-tuning, specify the `evaluation_strategy` parameter in your training arguments to report the evaluation metric at the end of each epoch:

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

### Trainer

Create a [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) object with your model, training arguments, training and test datasets, and evaluation function:

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.023443,0.561000
2,No log,1.211159,0.547000
3,No log,1.205772,0.579000


TrainOutput(global_step=375, training_loss=0.68470068359375, metrics={'train_runtime': 328.6198, 'train_samples_per_second': 9.129, 'train_steps_per_second': 1.141, 'total_flos': 1079836856672256.0, 'train_loss': 0.68470068359375, 'epoch': 3.0})

In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.187312,0.475000
2,No log,0.989896,0.578000
3,No log,1.016615,0.596000


TrainOutput(global_step=375, training_loss=1.0164366048177083, metrics={'train_runtime': 336.9983, 'train_samples_per_second': 8.902, 'train_steps_per_second': 1.113, 'total_flos': 789354427392000.0, 'train_loss': 1.0164366048177083, 'epoch': 3.0})

### ===My Notes=== ###

The rest of this notebook is another approach to fine-tuning using Pytorch.

### ===End of My Notes=== ###

## Additional resources

For more fine-tuning examples, refer to:

- [🤗 Transformers Examples](https://github.com/huggingface/transformers/tree/main/examples) includes scripts
  to train common NLP tasks in PyTorch and TensorFlow.

- [🤗 Transformers Notebooks](https://huggingface.co/docs/transformers/main/en/notebooks) contains various notebooks on how to fine-tune a model for specific tasks in PyTorch and TensorFlow.